In [ ]:
import numpy as np
import math
from scipy.special import gammaincc  
def linear_complexity_test(M, n, epsilon):
    K = 6
    pi = [0.01047, 0.03125, 0.12500, 0.50000, 0.25000, 0.06250, 0.020833]

    N = math.floor(n / M)

    # Allocate arrays
    B_ = np.zeros(M, dtype=int)
    C = np.zeros(M, dtype=int)
    P = np.zeros(M, dtype=int)
    T = np.zeros(M, dtype=int)
    nu = np.zeros(K + 1, dtype=float)

    print("-----------------------------------------------------")
    print("\tL I N E A R  C O M P L E X I T Y")
    print("-----------------------------------------------------")
    print(f"\tM (substring length)     = {M}")
    print(f"\tN (number of substrings) = {N}")
    print("-----------------------------------------------------")
    print("        F R E Q U E N C Y                            ")
    print("-----------------------------------------------------")
    print("  C0   C1   C2   C3   C4   C5   C6    CHI2    P-value")
    print("-----------------------------------------------------")
    print(f"\tNote: {n % M} bits were discarded!")

    for ii in range(N):
        B_.fill(0)
        C.fill(0)
        T.fill(0)
        P.fill(0)

        L = 0
        m = -1
        d = 0
        C[0] = 1
        B_[0] = 1

        # DETERMINE LINEAR COMPLEXITY
        N_ = 0
        while N_ < M:
            d = int(epsilon[ii * M + N_])
            for i in range(1, L + 1):
                d += C[i] * epsilon[ii * M + N_ - i]
            d = d % 2

            if d == 1:
                T[:] = C
                P.fill(0)

                for j in range(M):
                    if B_[j] == 1:
                        P[j + N_ - m] = 1

                C = (C + P) % 2

                if L <= N_ / 2:
                    L = N_ + 1 - L
                    m = N_
                    B_[:] = T

            N_ += 1

        # Compute mean and T_
        parity = (M + 1) % 2
        sign = -1 if parity == 0 else 1
        mean = M / 2.0 + (9.0 + sign) / 36.0 - (1.0 / (2 ** M)) * (M / 3.0 + 2.0 / 9.0)

        parity = M % 2
        sign = 1 if parity == 0 else -1
        T_ = sign * (L - mean) + 2.0 / 9.0

        if T_ <= -2.5:
            nu[0] += 1
        elif -2.5 < T_ <= -1.5:
            nu[1] += 1
        elif -1.5 < T_ <= -0.5:
            nu[2] += 1
        elif -0.5 < T_ <= 0.5:
            nu[3] += 1
        elif 0.5 < T_ <= 1.5:
            nu[4] += 1
        elif 1.5 < T_ <= 2.5:
            nu[5] += 1
        else:
            nu[6] += 1

    # Compute chi-square and p-value
    chi2 = sum((nu[i] - N * pi[i]) ** 2 / (N * pi[i]) for i in range(K + 1))
    p_value = gammaincc(K / 2.0, chi2 / 2.0)

    for i in range(K + 1):
        print(f"{int(nu[i]):4d}", end=" ")

    print(f"{chi2:9.6f} {p_value:9.6f}\n")
    print(f"{'FAILURE' if p_value < 0.01 else 'SUCCESS'}\tp_value = {p_value}")

    return p_value
